In [21]:
from IPython.display import HTML
from fastai.vision.all import *
from fastai.vision.widgets import *

## Chilean wildcats image classifier

<img src="./images/gato_andino.jpg" alt="Andino" style="width: 200px;"/>

Hello there!

This app is for classifying pictures of the following Chilean wildcats. 

1. [Gato Andino](https://en.wikipedia.org/wiki/Andean_mountain_cat)
1. [Gato de las pampas](https://en.wikipedia.org/wiki/Pampas_cat)
1. [Gato Huiña](https://en.wikipedia.org/wiki/Pampas_cat)

*A `resnet18` pretrained-model was fine-tuned using about 100 pictures of each wildcat.*

In [49]:
# load the model in learn_inf
path = Path()
path.ls(file_exts='.pkl')
learn_inf = load_learner(path/'wildcat_model.pkl')

In [13]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [14]:
# listener method:
btn_run = widgets.Button(description='Classify')

In [20]:
# listener method:                      
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl:
        display(img.to_thumb(128,128))
    pred, idx, probs = learn_inf.predict(img)
    display(probs[idx])
    #if probs[idx] < .5:
    #    lbl_pred.value = f'Sorry the model dont have a confident prediction about the picture ({probs[idx]*100:01f}%)'
    lbl_pred.value = f'Predicted {pred} with {probs[idx]*100:.01f}% certainty'
    
btn_run.on_click(on_click_classify)

In [18]:
VBox([
     widgets.Label('Upload your picture and press the classify button for the model prediction!'),
     btn_upload,
     btn_run,
     out_pl,
     lbl_pred
     ])